In [ ]:
import torchio as tio
from pathlib import Path
import numpy as np
from pydicom import dcmread
from matplotlib import pyplot as plt
import shutil
import ants
from itertools import chain
from nipype.interfaces.ants import Registration, ApplyTransforms
from matplotlib.backends.backend_pdf import PdfPages
from nipype.interfaces.fsl.maths import IsotropicSmooth
from nipype.interfaces.fsl.maths import Threshold, ApplyMask
from nipype.interfaces.fsl.utils import RobustFOV
from nipype.interfaces.fsl.preprocess import BET
import re
from sklearn.model_selection import train_test_split
import os
import pandas as pd
from detectron2.structures import BoxMode
from math import sqrt
import json
from math import floor, ceil

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
path_data = Path('../data/')
limit_g = -1
save_pdf = False
include_controls_in_data = False

np.set_printoptions(formatter={'float_kind':"{:.2f}".format})

detectron2_folder = path_data / "detectron2"
nnDetection_folder = path_data / "nnDetection/Task100_baseline"

for file in chain(detectron2_folder.glob("*/*/*"), nnDetection_folder.glob("raw_splitted/*/*")):
    os.remove(file)

In [ ]:
# Block: Loads dicom files and converts to nifti
print("converting dicom to nifti...")

limit = limit_g

def dcm_2_nii(pt):
    folder = re.search(r"(anonymized|controls)", str(pt))
    if folder is None:
        print(f"{folder} does not exist.")
        return

    out_folder = path_data / f'nii/{folder.group()}/{pt.name}'

    if not out_folder.exists():
        # iterate through each dicom folder
        for f in pt.iterdir():
            dcm_data = dcmread(next(f.iterdir()))
            out_name = 'CT' if dcm_data.Modality == "CT" else 'PET'
            to_file = out_folder / f'{out_name}.nii.gz'
            if not to_file.exists():
                to_file.parent.mkdir(parents=True, exist_ok=True)
                img = tio.ScalarImage(f)
                img.save(to_file)
                print(f'\t converted {out_name}')

for pt in chain(path_data.glob('anonymized/*'), path_data.glob('controls/*')):
    if limit == 0: break
    limit -= 1
    print(pt)
    dcm_2_nii(pt)

In [ ]:
# Block: Crops CT, finds mask for skull stripping (using smoothing and thresdhold) to apply to cropped CT
print("cropping and skull stripping CT...")

limit = limit_g

def crop_bet_CT(pt):
    if not pt.joinpath('CT.nii.gz').exists():
        print('\tCT nifty file does not exist.')
        return
    
    if pt.joinpath('ANAT_smoothed_0-100_BET_mask.nii.gz').exists():
        print('\tCT handled as outlier.')
        return

    # crop to HEAD
    if pt.joinpath('CT.nii.gz').exists() and not pt.joinpath('CT_crop.nii.gz').exists():
        crop = RobustFOV()
        crop.inputs.in_file = f"{pt}/CT.nii.gz"
        crop.inputs.out_roi = f"{pt}/CT_crop.nii.gz"
        crop.inputs.out_transform = f"{pt}/CT_crop_transform.mat"
        crop.run()
        print('\tcropped CT.')

    # smooth image
    if pt.joinpath('CT_crop.nii.gz').exists() and not pt.joinpath('ANAT_smoothed.nii.gz').exists():
        smoothing = IsotropicSmooth()
        smoothing.inputs.in_file = f"{pt}/CT_crop.nii.gz"
        smoothing.inputs.sigma = 1
        smoothing.inputs.out_file = f"{pt}/ANAT_smoothed.nii.gz"
        smoothing.run()
        print('\tapplied smooth for CT.')

    # threshold image
    if pt.joinpath('ANAT_smoothed.nii.gz').exists() and not pt.joinpath('ANAT_smoothed_0-100.nii.gz').exists():
        clamp = Threshold()
        clamp.inputs.in_file = f"{pt}/ANAT_smoothed.nii.gz"
        clamp.inputs.thresh = 0.0
        clamp.inputs.direction = 'below'
        clamp.inputs.out_file = f"{pt}/ANAT_smoothed_0.nii.gz"
        clamp.run()
        
        clamp = Threshold()
        clamp.inputs.in_file = f"{pt}/ANAT_smoothed_0.nii.gz"
        clamp.inputs.thresh = 100.0
        clamp.inputs.direction = 'above'
        clamp.inputs.out_file = f"{pt}/ANAT_smoothed_0-100.nii.gz"
        clamp.run()
        print('\tapplied threshold for CT.')
        
    # skull Strip
    if pt.joinpath('ANAT_smoothed_0-100.nii.gz').exists() and not pt.joinpath('ANAT_smoothed_0-100_BET.nii.gz').exists():
        bet = BET()
        bet.inputs.in_file = f"{pt}/ANAT_smoothed_0-100.nii.gz"
        bet.inputs.mask = True
        bet.inputs.frac = 0.1
        bet.inputs.out_file = f"{pt}/ANAT_smoothed_0-100_BET.nii.gz"
        bet.run()
        print('\tfound skull strip mask for CT.')

    # applied found mask to cropped CT (without smooth and threshold)
    if pt.joinpath('CT_crop.nii.gz').exists() and pt.joinpath('ANAT_smoothed_0-100_BET_mask.nii.gz').exists() and not pt.joinpath('CT_BET.nii.gz').exists():
        mask = ApplyMask()
        mask.inputs.in_file = f"{pt}/CT_crop.nii.gz"
        mask.inputs.mask_file = f"{pt}/ANAT_smoothed_0-100_BET_mask.nii.gz"
        mask.inputs.out_file = f"{pt}/CT_BET.nii.gz"
        mask.run()
        print('\tapplied skull strip mask on CT.')

for pt in path_data.glob('nii/*/*'):
    if limit == 0: break
    limit -= 1
    print(pt)
    crop_bet_CT(pt)

In [ ]:
# Block : Crops PET nifti file according to CP crop
print("cropping PET according to CT...")

limit = limit_g

def crop_PET_2_CT(pt):
    if pt.joinpath('PET.nii.gz').exists() and pt.joinpath('CT_crop.nii.gz').exists() and not pt.joinpath('PET_crop.nii.gz').exists():
        PET = tio.ScalarImage(f"{pt}/PET.nii.gz")
        CT_crop = tio.ScalarImage(f"{pt}/CT_crop.nii.gz")

        PET_np = PET.numpy()

        numSlices = int(np.round(CT_crop.shape[-1]*CT_crop.spacing[-1] / PET.spacing[-1]))
        PET_np = PET_np[:,:,:,-numSlices:]
        PET_np_affine = PET.affine.copy()
        PET_np_affine[2,3] = PET_np_affine[2,3] + PET.spacing[-1]*(PET.shape[-1]-(numSlices+1))
        PET_crop = tio.ScalarImage(tensor=PET_np, affine=PET_np_affine)
        PET_crop.save(f"{pt}/PET_crop.nii.gz")
        print('\tcropped PET.')

for pt in path_data.glob('nii/*/*'):
    if limit == 0: break
    limit -= 1
    print(pt)
    crop_PET_2_CT(pt)

In [ ]:
# Block: Registration between CT crop and PET crop
print("registering CT to PET...")

limit = limit_g
ct_to_pet = Registration()

ct_to_pet.inputs.metric = ["Mattes"]
ct_to_pet.inputs.metric_weight = [1]
ct_to_pet.inputs.smoothing_sigmas = [[3.0, 2.0, 1.0, 0.0]]
ct_to_pet.inputs.shrink_factors = [[8, 4, 2, 1]]
ct_to_pet.inputs.convergence_window_size = [10]
ct_to_pet.inputs.transforms = ["Rigid"]
ct_to_pet.inputs.number_of_iterations = [[1000, 500, 50, 100]]
ct_to_pet.inputs.initial_moving_transform_com = 0
ct_to_pet.inputs.transform_parameters = [(0.1,)]
ct_to_pet.inputs.radius_or_number_of_bins = [32]
ct_to_pet.inputs.num_threads = 6
ct_to_pet.inputs.winsorize_lower_quantile = 0.005
ct_to_pet.inputs.winsorize_upper_quantile = 0.995
ct_to_pet.inputs.use_histogram_matching = False
ct_to_pet.inputs.initialize_transforms_per_stage = False
ct_to_pet.inputs.write_composite_transform = True
ct_to_pet.inputs.collapse_output_transforms = True
ct_to_pet.inputs.verbose = True

def register_CT_2_PET(pt):
    if pt.joinpath('PET_crop.nii.gz').exists() and pt.joinpath("CT_BET.nii.gz").exists() and not pt.joinpath("Composite.h5").exists():
        ct_to_pet.inputs.fixed_image = f"{pt}/PET_crop.nii.gz"
        ct_to_pet.inputs.moving_image = f"{pt}/CT_BET.nii.gz"
        ct_to_pet.inputs.output_warped_image = f"{pt}/CT_BET_rslPET_crop.nii.gz"
        ct_to_pet.inputs.output_transform_prefix = f"{pt}/"
        ct_to_pet.run()
        print('\tregistered CT to PET.')

for pt in path_data.glob('nii/*/*'):
    if limit == 0: break
    limit -= 1
    print(pt)
    register_CT_2_PET(pt)

In [ ]:
# Block: ..
print("Resampling to 2mm isotropic...")

limit = limit_g
rsl = tio.Resample(2)

def resample_to_isotropic(pt):
    for file in ['CT_BET','PET_crop','ANAT_smoothed_0-100_BET_mask']:
        if pt.joinpath(f'{file}.nii.gz').exists() and not pt.joinpath(f"2mm{file}.nii.gz").exists():
            if file.endswith('mask'):
                img = tio.LabelMap(pt.joinpath(f'{file}.nii.gz'))
            else:
                img = tio.ScalarImage(pt.joinpath(f'{file}.nii.gz'))
            rslImg = rsl(img)
            rslImg.save(pt.joinpath(f"2mm{file}.nii.gz"))
            print('\tsaved 2mm for',file)

for pt in path_data.glob('nii/*/*'):
    if limit == 0: break
    limit -= 1
    print(pt)
    resample_to_isotropic(pt)

In [ ]:
# Block: ...
print("Warping PET to CT...")

limit = limit_g

at = ApplyTransforms()
at.inputs.dimension = 3
at.inputs.interpolation = 'Linear'
at.inputs.default_value = 0
at.inputs.invert_transform_flags = True

def warp_PET_2_CT(pt):
    if pt.joinpath('2mmPET_crop.nii.gz').exists() and pt.joinpath("2mmCT_BET.nii.gz").exists() and pt.joinpath("Composite.h5").exists() and not pt.joinpath("2mmPET_crop_rsl2mmCT_BET.nii.gz").exists():
        at.inputs.input_image = str(pt.joinpath('2mmPET_crop.nii.gz'))
        at.inputs.reference_image = str(pt.joinpath('2mmCT_BET.nii.gz'))
        at.inputs.output_image = str(pt.joinpath('2mmPET_crop_rsl2mmCT_BET.nii.gz'))
        at.inputs.transforms = str(pt.joinpath('Composite.h5'))
        at.run()
        print('\twarped PET.')

for pt in path_data.glob('nii/*/*'):
    if limit == 0: break
    limit -= 1
    print(pt)
    warp_PET_2_CT(pt)

In [ ]:
limit = limit_g
mask = ApplyMask()

def bet_PET(pt):
    if pt.joinpath('2mmPET_crop_rsl2mmCT_BET.nii.gz').exists() and pt.joinpath("2mmANAT_smoothed_0-100_BET_mask.nii.gz").exists() and not pt.joinpath("2mmPET_BET_rsl2mmCT_BET.nii.gz").exists():
        mask.inputs.in_file = f"{pt}/2mmPET_crop_rsl2mmCT_BET.nii.gz"
        mask.inputs.mask_file = f"{pt}/2mmANAT_smoothed_0-100_BET_mask.nii.gz"
        mask.inputs.out_file = f"{pt}/2mmPET_BET_rsl2mmCT_BET.nii.gz"
        mask.run()
        print('\tskull stripped PET.')

for pt in path_data.glob('nii/*/*'):
    if limit == 0: break
    limit -= 1
    print(pt)
    bet_PET(pt)

In [ ]:
# Block: saves registrations as pdf
def plot_registration(pt, pdf, ctmin=0, ctmax=300):
    if pt.joinpath('2mmPET_BET_rsl2mmCT_BET.nii.gz').exists() and pt.joinpath('2mmCT_BET.nii.gz').exists():
        PET = ants.image_read(f"{pt}/2mmPET_BET_rsl2mmCT_BET.nii.gz")
        CT = ants.image_read(f"{pt}/2mmCT_BET.nii.gz")
        
        fig, ax = plt.subplots(1,2,dpi=300,figsize=(20,10))
        fig.suptitle(str(pt))

        PET_slice = np.argmax(np.sum(PET.numpy(),(0,1)))
        CT_slice = np.argmax(np.sum(CT.numpy(),(0,1)))

        ax[0].imshow(PET[:,:,PET_slice], alpha=0.5)
        ax[0].set_title('PET 2mm BET resampled to CT 2mm BET')

        ax[1].imshow(CT[:,:,PET_slice], alpha=0.5)
        ax[1].set_title('CT 2mm BET')

        pdf.savefig(fig)

if save_pdf:
    print("saving registrations to pdf...")

    pdf = PdfPages(f"{path_data}/registrations_anonymized.pdf")

    for pt in path_data.glob('nii/anonymized/*'):
        plot_registration(pt, pdf)

    pdf.close()

    pdf = PdfPages(f"{path_data}/registrations_controls.pdf")

    for pt in path_data.glob('nii/controls/*'):
        plot_registration(pt, pdf)

    pdf.close()

In [ ]:
# split data into train and test set

def build_record(category):
    record = {}

    for pt in path_data.glob(f"{category}/*"):
        rel_pt = path_data / f"nii/{category}/{str(pt)[-8:]}"

        if not rel_pt.exists():
            print(f"\tNo matching patient found {rel_pt}.")
            continue

        for folder in pt.iterdir():
            ds = dcmread(next(folder.iterdir()))
            # 0=Bisbebjerg, 1=Herlev/Gentofte/Default
            record[rel_pt] = f"{category[0]}0" if str(ds.InstitutionName) == "Bispebjerg Hospital" else f"{category[0]}1"
            break

    return record

def split_data(include_controls):
    anonymized_record = build_record('anonymized')
    record_pts = list(anonymized_record.keys())
    record_labels = list(anonymized_record.values())

    if (include_controls):
        controls_record = build_record('controls')
        record_pts = record_pts + list(controls_record.keys())
        record_labels = record_labels + list(controls_record.values())

    pts = np.array(record_pts)
    labels = np.array(record_labels)

    pts_train, pts_test = train_test_split(
        pts, test_size=0.1, random_state=42, stratify=labels)

    return (pts_train, pts_test)

(pts_train, pts_test) = split_data(include_controls_in_data)

In [ ]:
# structure data for models

def sphere(shape, radius, position):
    assert len(position) == len(shape)
    n = len(shape)
    semisizes = (radius,) * len(shape)

    grid = [slice(-x0, dim - x0) for x0, dim in zip(position, shape)]
    position = np.ogrid[grid]
    arr = np.zeros(shape, dtype=float)
    for x_i, semisize in zip(position, semisizes):
        arr += (x_i / semisize) ** 2

    return arr <= 1.0

def structure_data(data, df, train, model):
    limit = -1
    nnDetection_category = "Tr" if train else "Ts"
    detectron2_category = "train" if train else "test"
    records = {'axial':[], 'coronal':[], 'sagital':[]}

    for pt in data:
        if limit == 0:
            break
        limit -= 1

        pt_name = str(pt)[-8:]

        print(f"\ninspecting patient {pt_name}")

        PET_path = pt / "2mmPET_BET_rsl2mmCT_BET.nii.gz"
        CT_path = pt / "2mmCT_BET.nii.gz"

        rsl = tio.Resample(0.8)

        PET = rsl(tio.ScalarImage(PET_path)).numpy()[0]
        CT = rsl(tio.ScalarImage(CT_path)).numpy()[0]

        if not PET_path.exists() and CT_path.exists():
            print(f"files not found for patient {pt_name}... continuing to next patient")
            continue

        print("\tfiles found.")

        # get annotations for patient
        annotations = df[df['ID'] == pt_name]

        if model == 'detectron2':
            # build annotations for detectron2
            if len(annotations) == 0:
                print('handle controls')
            else:
                #annotations_d2 = []

                for ia, annotation in enumerate(annotations.iloc):
                    radius = ((float(annotation['diameter_in_cm'].replace(',','.')) / 2) * 10) / 2 # cm to mm to voxel
                    rtss = np.array(
                        [
                            annotation.voxel_x,
                            annotation.voxel_y,
                            annotation.voxel_z,
                            radius,
                            radius,
                            radius
                        ]
                    )

                    radius = round(radius * 0.2)

                    # axial
                    #for z in range(PET.shape(3)):
                    for z in range(floor(annotation.voxel_z - radius), ceil(annotation.voxel_z + radius + 1)):
                        axial = np.zeros((PET.shape[0], PET.shape[1], 3), dtype='float32')

                        axial[:,:,0] = PET[:,:,z]
                        axial[:,:,1] = CT[:,:,z]

                        np.save(detectron2_folder / f"axial/{detectron2_category}/{pt_name}_{z:04}", axial)

                        bbox = np.concatenate((rtss[:2], rtss[3:5]))
                        annotations_d2 = [
                                {
                                    "bbox": bbox.tolist(),
                                    "bbox_mode": BoxMode.XYWH_ABS,
                                    "category_id": 0
                                }
                            ]

                        data = {
                            "file_name": f"{pt_name}_{z:04}.npy",
                            "image_id": f"{pt_name}_{z:04}",
                            "height": axial.shape[0],
                            "width": axial.shape[1],
                            "annotations": annotations_d2
                        }

                        records['axial'].append(data)
                    
                    # coronal
                    #for x in range(PET.shape(3)):
                    for x in range(floor(annotation.voxel_x - radius), ceil(annotation.voxel_x + radius + 1)):
                        coronal = np.zeros((PET.shape[1], PET.shape[2], 3), dtype='float32')

                        coronal[:,:,0] = PET[x,:,:]
                        coronal[:,:,1] = CT[x,:,:]

                        np.save(detectron2_folder / f"coronal/{detectron2_category}/{pt_name}_{z:04}", coronal)

                        bbox = np.concatenate((rtss[1:3], rtss[4:]))
                        annotations_d2 = [
                                {
                                    "bbox": bbox.tolist(),
                                    "bbox_mode": BoxMode.XYWH_ABS,
                                    "category_id": 0
                                }
                            ]

                        data = {
                            "file_name": f"{pt_name}_{z:04}.npy",
                            "image_id": f"{pt_name}_{z:04}",
                            "height": coronal.shape[1],
                            "width": coronal.shape[2],
                            "annotations": annotations_d2
                        }

                        records['coronal'].append(data)
                    
                    # sagital
                    #for y in range(PET.shape(3)):
                    for y in range(floor(annotation.voxel_y - radius), ceil(annotation.voxel_y + radius + 1)):
                        sagital = np.zeros((PET.shape[0], PET.shape[2], 3), dtype='float32')

                        sagital[:,:,0] = PET[:,y,:]
                        sagital[:,:,1] = CT[:,y,:]

                        np.save(detectron2_folder / f"sagital/{detectron2_category}/{pt_name}_{z:04}", sagital)

                        bbox = np.array([rtss[0], rtss[2], rtss[3], rtss[5]])
                        annotations_d2 = [
                                {
                                    "bbox": bbox.tolist(),
                                    "bbox_mode": BoxMode.XYWH_ABS,
                                    "category_id": 0
                                }
                            ]

                        data = {
                            "file_name": f"{pt_name}_{z:04}.npy",
                            "image_id": f"{pt_name}_{z:04}",
                            "height": sagital.shape[0],
                            "width": sagital.shape[2],
                            "annotations": annotations_d2
                        }

                        records['sagital'].append(data)

            print("\tdetectron2 annotations built")
        elif model == 'nnDetection':
            shutil.copy(PET_path, nnDetection_folder / "raw_splitted" / f"images{nnDetection_category}/{pt_name}_0000.nii.gz")
            shutil.copy(CT_path, nnDetection_folder / "raw_splitted" / f"images{nnDetection_category}/{pt_name}_0001.nii.gz")

            print("\tnnDetection files copied")

            CT = tio.ScalarImage(CT_path)
            mask = np.zeros_like(CT.numpy()[0])
            instances = {}

            for ia, annotation in enumerate(annotations.iloc):
                annotation_label = ia+1
                if not (str(annotation_label) in instances):
                    instances[str(annotation_label)] = 0

                tumor_center = np.rint(np.array([annotation.voxel_x, annotation.voxel_y, annotation.voxel_z]))
                tumor_radius = (float(annotation['diameter_in_cm'].replace(',','.')) / 2) * 10 / 2 # cm to mm to voxels

                arr = sphere(mask.shape, tumor_radius, tumor_center)
                mask[arr] = annotation_label
            
            tio.LabelMap(tensor=np.expand_dims(mask, 0), affine=CT.affine).save(nnDetection_folder / f"raw_splitted/labels{nnDetection_category}/{pt_name}.nii.gz")

            print("\tnnDetection annotations built")
            
            with open(nnDetection_folder / f"raw_splitted/labels{nnDetection_category}/{pt_name}.json", 'w') as json_file:
                json_data = json.dumps({"instances": instances}, indent=4)
                json_file.write(json_data)

            print("\tnnDetection annotations written")
        else:
            print('model not specified')
            return
    
    with open(detectron2_folder / f"axial/{detectron2_category}/dataset.json", "w") as json_file:
        json_data = json.dumps(records['axial'], indent=4)
        json_file.write(json_data)
    
    with open(detectron2_folder / f"coronal/{detectron2_category}/dataset.json", "w") as json_file:
        json_data = json.dumps(records['coronal'], indent=4)
        json_file.write(json_data)
    
    with open(detectron2_folder / f"sagital/{detectron2_category}/dataset.json", "w") as json_file:
        json_data = json.dumps(records['sagital'], indent=4)
        json_file.write(json_data)
    
    print("\tdetectron2 annotations written")

df = pd.read_pickle(path_data / "registered_reference_markings_0p8mm.pkl")
model = 'detectron2'

structure_data(pts_train, df, True, model)
structure_data(pts_test, df, False, model)